# Project EdgeAI

## Import library to get data

In [27]:
import requests, os, shutil

listQuery = ["Emberiza citrinella", "Alauda arvensis", "Cuculus canorus" ] #, "Emberiza cirlus", "Tyto alba", "Falco tinnunculus", "Sylvia borin", "Muscicapa striata", "Tachybaptus ruficollis", "Delichon urbicum"]

## Import data from API

In [28]:
url = "https://xeno-canto.org/api/2/recordings?query="

def enregistrer_mp3(url_api, nom_fichier):
    reponse = requests.get(url_api, stream=True)
    
    if reponse.status_code == 200:
        with open(nom_fichier, 'wb') as fichier_audio:
            reponse.raw.decode_content = True
            shutil.copyfileobj(reponse.raw, fichier_audio)
        print("Audio enregistré avec succès sous le nom :", nom_fichier)
    else:
        print("Erreur lors de la récupération des données audio depuis l'API")

def getData(query):
    url_api = url + query
    response = requests.get(url_api)

    if response.status_code == 200:
        contenu = response.json()["recordings"]
        pathToDir = "./data/" + query

        if not os.path.exists(pathToDir):
            os.mkdir(pathToDir)

            for _, element in zip(range(250), contenu):
                elementUrl = element["file"]
                elementFileName = element["file-name"]
                storeFile = pathToDir + "/" + elementFileName
                
                if not os.path.exists(storeFile):
                    try:
                        enregistrer_mp3(elementUrl, storeFile)
                    except: # Pour les problèmes de noms de fichiers
                        pass

for query in listQuery:
    getData(query)

## Import library to train model

In [29]:
from sklearn import model_selection
from pathlib import Path
import librosa
import copy
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.activations import softmax

from keras.layers import Input, Conv1D, MaxPool1D, Flatten, Dense, AvgPool1D, Activation
from keras.utils import get_file
from keras.utils import to_categorical

dataset_dir = Path('data')

## Load all sound in X and Y

In [30]:
x = []
y = []

def slice_audio_with_trim(file_path, index, duration=1):
    audio_data, sampling_rate = librosa.load(file_path, sr=None)
    non_silent_audio, _ = librosa.effects.trim(audio_data)
    num_slices = int(len(non_silent_audio) / sampling_rate / duration)

    # Diviser l'audio en fonction de duration
    for i in range(num_slices):
        start = i * sampling_rate * duration
        end = start + sampling_rate * duration
        slice_data = non_silent_audio[start:end]

        data = slice_data.astype(np.float32)
        data.resize((16000, 1))

        x.append(data)
        y.append(index)

def getAllDataClasses(directory, index):
    for recording in dataset_dir.glob(f'{directory}/*'):
        slice_audio_with_trim(str(recording) ,index)

def loadAllData():
    for i in range(len(listQuery)):
        getAllDataClasses(listQuery[i], i)

loadAllData()

In [31]:
u, indexes, counts = np.unique(y, return_index=True, return_counts=True)
min = np.min(counts)
nbrClasses = len(u)

indices = np.random.permutation(min)

newX = []
newY = []

for i in range(nbrClasses):
    sousSetX = []
    sousSetY = []
    
    if i == nbrClasses - 1:
        sousSetX = x[indexes[i]:]
        sousSetY = y[indexes[i]:]
    else:
        sousSetX = x[indexes[i] : indexes[i + 1]]
        sousSetY = y[indexes[i] : indexes[i + 1]]
    
    x_selectionne = [sousSetX[j] for j in indices]
    y_selectionne = [sousSetY[j] for j in indices]

    for x_s, y_s in zip(x_selectionne, y_selectionne):
        newX.append(x_s)
        newY.append(y_s)

u, indexes, counts = np.unique(newY, return_index=True, return_counts=True)
print(u)
print(indexes)
print(counts)

[0 1 2]
[   0 1121 2242]
[1121 1121 1121]


In [96]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(newX, newY, test_size=0.2)

x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = to_categorical(np.array(y_train))
y_test = to_categorical(np.array(y_test))

perms = np.random.permutation(len(y_test))[0:250]

x_test_250 = x_test[perms]
y_test_250 = y_test[perms]

np.savetxt('x_test_gsc_250.csv', x_test_250.reshape((x_test_250.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_test_gsc_250.csv', y_test_250, delimiter=',', fmt='%s')


In [97]:
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

(2690, 16000, 1) (673, 16000, 1)
(2690, 3) (673, 3)


In [99]:
model = Sequential()
model.add(Input(shape=(16000, 1)))
model.add(MaxPool1D(pool_size=12))
model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(filters=30, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(AvgPool1D())
model.add(Conv1D(filters=4, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=8))
model.add(Conv1D(filters=4, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=3))
model.add(Activation('softmax')) # SoftMax activation needs to be separate from Dense to remove it later on
# EXPLORE Learning Rate
opt = tf.keras.optimizers.Adam(learning_rate=10e-4)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 max_pooling1d_58 (MaxPoolin  (None, 1333, 1)          0         
 g1D)                                                            
                                                                 
 conv1d_48 (Conv1D)          (None, 1331, 16)          64        
                                                                 
 max_pooling1d_59 (MaxPoolin  (None, 332, 16)          0         
 g1D)                                                            
                                                                 
 conv1d_49 (Conv1D)          (None, 330, 30)           1470      
                                                                 
 max_pooling1d_60 (MaxPoolin  (None, 82, 30)           0         
 g1D)                                                            
                                                     

In [100]:
model.fit(x_train, y_train, epochs=50, batch_size=268, validation_data=(x_test, y_test))
# model.fit(x_train, y_train, epochs=25, batch_size=268, validation_data=(x_test, y_test))

Epoch 1/50
11/11 [==============================] - 4s 181ms/step - loss: 1.0982 - categorical_accuracy: 0.3673 - val_loss: 1.0975 - val_categorical_accuracy: 0.3105
Epoch 2/50
11/11 [==============================] - 2s 141ms/step - loss: 1.0957 - categorical_accuracy: 0.3390 - val_loss: 1.0952 - val_categorical_accuracy: 0.3120
Epoch 3/50
11/11 [==============================] - 1s 126ms/step - loss: 1.0913 - categorical_accuracy: 0.3491 - val_loss: 1.0908 - val_categorical_accuracy: 0.3105
Epoch 4/50
11/11 [==============================] - 2s 141ms/step - loss: 1.0834 - categorical_accuracy: 0.3632 - val_loss: 1.0835 - val_categorical_accuracy: 0.3299
Epoch 5/50
11/11 [==============================] - 2s 141ms/step - loss: 1.0703 - categorical_accuracy: 0.3717 - val_loss: 1.0721 - val_categorical_accuracy: 0.4012
Epoch 6/50
11/11 [==============================] - 1s 138ms/step - loss: 1.0527 - categorical_accuracy: 0.4372 - val_loss: 1.0609 - val_categorical_accuracy: 0.4205
Epoc

In [101]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

22/22 - 0s - loss: 0.4777 - categorical_accuracy: 0.8425 - 321ms/epoch - 15ms/step
22/22 [==============================] - 0s 11ms/step
tf.Tensor(
[[174  54   5]
 [ 16 208   6]
 [ 20   5 185]], shape=(3, 3), dtype=int32)


In [102]:
model.evaluate(x_test_250, y_test_250, verbose=2)
pred_test_250 = model.predict(x_test_250)
print(tf.math.confusion_matrix(y_test_250.argmax(axis=1), pred_test_250.argmax(axis=1)))

8/8 - 0s - loss: 0.4748 - categorical_accuracy: 0.8440 - 149ms/epoch - 19ms/step
8/8 [==============================] - 0s 11ms/step
tf.Tensor(
[[67 19  4]
 [ 4 83  2]
 [ 9  1 61]], shape=(3, 3), dtype=int32)


In [88]:
model.save('project.h5')

In [103]:
model.save('project.h5')
if isinstance(model.layers[-1], Activation) and model.layers[-1].activation == softmax:
    model = tf.keras.Model(model.input, model.layers[-2].output, name=model.name)
else:
    print('Error: last layer is not SoftMax Activation')

In [104]:
%pip install qualia_codegen_core
import qualia_codegen_core
from qualia_codegen_core.graph.KerasModelGraph import KerasModelGraph
from qualia_codegen_core.graph.Quantization import Quantization
from qualia_codegen_core.graph.RoundMode import RoundMode

from importlib.resources import files
main_path = str((files('qualia_codegen_core.examples')/'Linux'/'main.cpp').resolve())

Note: you may need to restart the kernel to use updated packages.


In [105]:
modelgraph = KerasModelGraph(model).convert()
print(modelgraph)

—————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
Inputs                                           | Layer                                            | Outputs                                          | Input shape                                      | Output shape                                    
—————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
                                                 | input_13                                         | max_pooling1d_58                                 | (1, 16000, 1)                                    | ((1, 16000, 1),)                   

In [106]:
float_modelgraph = copy.deepcopy(modelgraph)

# layer quantization annotations for float32
for node in float_modelgraph.nodes:
    # No scale factor if not fixed-point quantization on integers
    node.q = Quantization(
            number_type=float,
            width=32,
            long_width=32,
            weights_scale_factor=0,
            output_scale_factor=0,
            weights_round_mode=RoundMode.NONE,
            output_round_mode=RoundMode.NONE,
            )

float_res = qualia_codegen_core.Converter(output_path=Path('gsc_output_floating')).convert_model(float_modelgraph)

with open('gsc_model_floating.h', 'w') as f:
    f.write(float_res)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\average_pooling1d
......vars
...layers\conv1d
......vars
.........0
.........1
...layers\conv1d_1
......vars
.........0
.........1
...layers\conv1d_2
......vars
.........0
.........1
...layers\conv1d_3
......vars
.........0
.........1
...layers\dense
......vars
.........0
.........1
...layers\flatten
......vars
...layers\input_layer
......vars
...layers\max_pooling1d
......vars
...layers\max_pooling1d_1
......vars
...layers\max_pooling1d_2
......vars
...layers\max_pooling1d_3
......vars
...layers\max_pooling1d_4
......vars
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2024-04-28 14:36:15         5312
metadata.json                                  2024-04-28 14:36:15           64
variables.h5                                   2024-04-28 14:36:15        39232
Keras model archive loading:
File Name 

Graphviz not available
Activation function TActivation.SOFTMAX not supported
ModelGraph validation failed


TypeError: write() argument must be str, not None

Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\average_pooling1d
......vars
...layers\conv1d
......vars
.........0
.........1
...layers\conv1d_1
......vars
.........0
.........1
...layers\conv1d_2
......vars
.........0
.........1
...layers\conv1d_3
......vars
.........0
.........1
...layers\dense
......vars
.........0
.........1
...layers\flatten
......vars
...layers\input_layer
......vars
...layers\max_pooling1d
......vars
...layers\max_pooling1d_1
......vars
...layers\max_pooling1d_2
......vars
...layers\max_pooling1d_3
......vars
...layers\max_pooling1d_4
......vars
...vars


In [107]:
!g++ -std=c++17 -Wall -Wextra -pedantic -Ofast -o gsc_fixed -include gsc_output_fixed/include/defines.h -Igsc_output_fixed/include gsc_output_fixed/model.c {main_path}

cc1plus.exe: fatal error: gsc_output_fixed/model.c: No such file or directory
compilation terminated.
<command-line>: fatal error: gsc_output_fixed/include/defines.h: No such file or directory
compilation terminated.


In [108]:
fixed_modelgraph = copy.deepcopy(modelgraph)

# layer quantization annotations for int16 Q9.7
for node in fixed_modelgraph.nodes:
    node.q = Quantization(
            number_type=int,
            width=16,
            long_width=32,
            weights_scale_factor=7,
            output_scale_factor=7,
            weights_round_mode=RoundMode.FLOOR,
            output_round_mode=RoundMode.FLOOR,
            )

fixed_res = qualia_codegen_core.Converter(output_path=Path('gsc_output_fixed')).convert_model(fixed_modelgraph)

with open('gsc_model_fixed.h', 'w') as f:
    f.write(fixed_res)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\average_pooling1d
......vars
...layers\conv1d
......vars
.........0
.........1
...layers\conv1d_1
......vars
.........0
.........1
...layers\conv1d_2
......vars
.........0
.........1
...layers\conv1d_3
......vars
.........0
.........1
...layers\dense
......vars
.........0
.........1
...layers\flatten
......vars
...layers\input_layer
......vars
...layers\max_pooling1d
......vars
...layers\max_pooling1d_1
......vars
...layers\max_pooling1d_2
......vars
...layers\max_pooling1d_3
......vars
...layers\max_pooling1d_4
......vars
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2024-04-28 14:36:20         5312
metadata.json                                  2024-04-28 14:36:20           64
variables.h5                                   2024-04-28 14:36:20        39232
Keras model archive loading:
File Name 

Graphviz not available


In [109]:
!g++ -std=c++17 -Wall -Wextra -pedantic -Ofast -o gsc_fixed -include gsc_output_fixed/include/defines.h -Igsc_output_fixed/include gsc_output_fixed/model.c {main_path}
!./gsc_fixed x_test_gsc_250.csv y_test_gsc_250.csv

'.' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.
